In [1]:

from pyexpat import model
from tkinter.tix import Tree
from caption import *
from ipywidgets import interact, interactive, interact_manual
import ipywidgets as widgets
from IPython.display import display
import os

In [10]:
images ={}
images_dir ='images'
for i in os.listdir(images_dir):
    name = i.split(".")
    images[name[0]]=os.path.join(images_dir,i)
print(images)

{'': 'images\\.ipynb_checkpoints', 'bongdaVN': 'images\\bongdaVN.jpg', 'bongdaVN_2': 'images\\bongdaVN_2.jpg', 'dog_1': 'images\\dog_1.jpg', 'dog_2': 'images\\dog_2.jpg', 'dog_3': 'images\\dog_3.jpg', 'maybay': 'images\\maybay.jpg', 'maybay_2': 'images\\maybay_2.jpg', 'pizza': 'images\\pizza.jpg', 'voi': 'images\\voi.jpg'}


In [7]:
smooth = True
model_type={
    "coco":"coco",
    "flickr30k":"flickr30k"
}


In [8]:
def demo(image_path,model_wordMap,beam_size):
    smooth=True
    coco = '\BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq_review_tokenize_vi.pth.tar'
    flickr30k='\BEST_checkpoint_flickr30k_5_cap_per_img_5_min_word_freq_review_tokenize_vi.pth.tar'
    coco_word_map ='\WORDMAP_coco_5_cap_per_img_5_min_word_freq_review_tokenize_vi.json'
    flickr30k_word_map="\WORDMAP_flickr30k_5_cap_per_img_5_min_word_freq_review_tokenize_vi.json"
    img_path = images[image_path]
    if model_type[model_wordMap]=="flickr30k":
        model_path = '.\model+word_map' + flickr30k
        word_map_path ='.\model+word_map' + flickr30k_word_map
    else:
        model_path = '.\model+word_map' + coco
        word_map_path ='.\model+word_map' + coco_word_map
    

    checkpoint = torch.load(model_path, map_location=str(device))
    decoder = checkpoint['decoder']
    decoder = decoder.to(device)
    decoder.eval()
    encoder = checkpoint['encoder']
    encoder = encoder.to(device)
    encoder.eval()

    # Load word map (word2ix)
    with open(word_map_path, 'r') as j:
        word_map = json.load(j)
    rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

    # Encode, decode with attention and beam search
    seq, alphas = caption_image_beam_search(encoder, decoder, img_path, word_map, beam_size=beam_size)
    alphas = torch.FloatTensor(alphas)

    # Visualize caption and attention of best sequence
    visualize_att(img_path, seq, alphas, rev_word_map, smooth=smooth)

In [9]:
    int_slider_widget = widgets.IntSlider(min=1,max=5)
    w = interactive(demo,image_path=list(images.keys()),model_wordMap=list(model_type.keys()),beam_size=int_slider_widget)
    display(w)

interactive(children=(Dropdown(description='image_path', options=('', 'bongdaVN', 'bongdaVN_2', 'dog_1', 'dog_…